In [ ]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import *
import sympy as sp
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import jax
jax.config.update("jax_enable_x64", True)

### THE ITERATION PARAMETERs
#physical parameters
delta = 0.012 #interface width
Lambda = jnp.sqrt(2)*delta/4
#g_ca = [2e-3, 3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3]
g_ca = [3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 1.2e-2, 1.3e-2, 1.4e-2, 1.5e-2, 1.7e-2]
gamma_bc = 1e-2
gamma_ab = 1e-2  

image_num = 50
N = 256 #num of nodes
r = jnp.linspace(0, 1, N+1)
r_mid = (r[1:] + r[:-1]) / 2
dr = r_mid[1] - r_mid[0]

In [ ]:
import sympy as sp

def cal_theoretical_angle(g_ca):
    gamma_ab = 1e-2
    gamma_bc = 1e-2
    x, y = sp.symbols('x y')
    eq1 = g_ca*sp.cos(x) + gamma_bc*sp.cos(y) - gamma_ab
    eq2 = g_ca*sp.sin(x) - gamma_bc*sp.sin(y)
    solution = sp.solve([eq1, eq2], (x, y))
    angle = solution[-1]
    return angle

def cal_shapefactor1(Angle):
    theta = float(Angle[0])
    phi = float(Angle[1])
    s1 = theta + phi*(jnp.power(jnp.sin(theta), 2)/jnp.power(jnp.sin(phi), 2)) - (jnp.sin(theta)/jnp.sin(phi))*jnp.sin(theta+phi)
    S = s1/jnp.pi
    return S

def cal_shapefactor2(Angle):
    theta = float(Angle[0])
    phi = float(Angle[1])
    s2 = phi + theta*(jnp.power(jnp.sin(phi), 2)/jnp.power(jnp.sin(theta), 2)) - (jnp.sin(phi)/jnp.sin(theta))*jnp.sin(phi+theta)
    S = s2/jnp.pi
    return S

sf = []
for gamma_ca in g_ca:
    angle = cal_theoretical_angle(g_ca=gamma_ca)
    if gamma_ca < 1e-2: sf.append(float(cal_shapefactor1(Angle=angle)))
    else: sf.append(float(cal_shapefactor2(Angle=angle)))

print(sf)
#sf = [0.468097, 0.4573867, 0.44661, 0.435748, 0.4247799, 0.413684]

In [ ]:

def cal_gradsquare(x, nn=0, corrected=False):

    if corrected: k = 2*jnp.pi*jnp.fft.fftfreq(N-2*nn, d=dr)
    else: k = 2*jnp.pi*jnp.fft.fftfreq(N, d=dr)
    kx, ky = jnp.meshgrid(k, k)
    k_square = kx**2 + ky**2

    xh = jnp.fft.fft2(x)
    p = jnp.fft.ifft2(k_square*xh).real
    return x*p


def main(case, corrected=False, nn=0):
    eb = []
    for gamma_ca in tqdm(g_ca):
        if gamma_ca > 1e-2:
            Ac = (9*gamma_ca + 12*(gamma_bc - gamma_ab))/(4*jnp.sqrt(2)*Lambda)
            Aa = (9*gamma_ca - 12*(gamma_bc - gamma_ab))/(4*jnp.sqrt(2)*Lambda)
            Ab = 3*jnp.sqrt(2)*gamma_bc/Lambda - 4*Ac/3 - Aa/3 
        else:
            Ab = jnp.sqrt(2)*(9*gamma_bc + 12*(gamma_ab - gamma_ca))/(8*Lambda)
            Ac = jnp.sqrt(2)*(9*gamma_bc - 12*(gamma_ab - gamma_ca))/(8*Lambda)
            Aa = 3*jnp.sqrt(2)*gamma_ab/Lambda - 4*Ab/3 - Ac/3
        A = jnp.array([Aa, Ab, Ac])
        C = jnp.max(A + jnp.roll(A, 1))/6

       
        u = jnp.load(f"Nucleation_ZTS/TernaryMix/2d/Output/AsymmetricCase/Data-InterfaceWidth-0.012/gamma_ca-{gamma_ca}/{case}/concentration.npy")
        if not corrected:
            ua = u[0]
            ub = u[1]
            uc = u[2]
            
            bulk_term = Aa*jnp.power(ua, 2)*jnp.power(ua-1, 2) + Ab*jnp.power(ub, 2)*jnp.power(ub-1, 2) + Ac*jnp.power(uc, 2)*jnp.power(uc-1, 2) + C*(jnp.power(ua,2)*jnp.power(ub, 2) + jnp.power(ua,2)*jnp.power(uc,2) + jnp.power(uc,2)*jnp.power(ub,2) + jnp.power(1-ua,2)*jnp.power(1-ub,2)*jnp.power(1-uc,2))
            
            Wa = jnp.power(Lambda, 2)*(Aa + C)
            Wb = jnp.power(Lambda, 2)*(Ab + C)
            Wc = jnp.power(Lambda, 2)*(Ac + C)

            interface_term = Wa*cal_gradsquare(ua)/2 + Wb*cal_gradsquare(ub)/2 + Wc*cal_gradsquare(uc)/2

            G = jnp.mean(bulk_term + interface_term, axis=(1,2))
            critical_index = jnp.argmax(G)
            barrier = G[critical_index] - G[0]
            eb.append(float(barrier))
        else:
            nn = nn
            #ua = u[0, :, nn:N-nn+1, nn:N-nn+1]
            #ub = u[1, :, nn:N-nn+1, nn:N-nn+1]
            #uc = u[2, :, nn:N-nn+1, nn:N-nn+1]
            ua = u[0, :, nn:N-nn, nn:N-nn]
            ub = u[1, :, nn:N-nn, nn:N-nn]
            uc = u[2, :, nn:N-nn, nn:N-nn]

            bulk_term = Aa*jnp.power(ua, 2)*jnp.power(ua-1, 2) + Ab*jnp.power(ub, 2)*jnp.power(ub-1, 2) + Ac*jnp.power(uc, 2)*jnp.power(uc-1, 2) + C*(jnp.power(ua,2)*jnp.power(ub, 2) + jnp.power(ua,2)*jnp.power(uc,2) + jnp.power(uc,2)*jnp.power(ub,2) + jnp.power(1-ua,2)*jnp.power(1-ub,2)*jnp.power(1-uc,2))
            
            Wa = jnp.power(Lambda, 2)*(Aa + C)
            Wb = jnp.power(Lambda, 2)*(Ab + C)
            Wc = jnp.power(Lambda, 2)*(Ac + C)

            interface_term = Wa*cal_gradsquare(ua, nn=nn, corrected=True)/2 + Wb*cal_gradsquare(ub, nn=nn, corrected=True)/2 + Wc*cal_gradsquare(uc, nn=nn, corrected=True)/2

            G = jnp.mean(bulk_term + interface_term, axis=(1,2))
            critical_index = jnp.argmax(G)
            barrier = G[critical_index] - G[0]
            eb.append(float(barrier))

    return eb

eb_het = main(case="heterogeneous", corrected=True, nn=6)
eb_hom = main(case="homogeneous", corrected=False, nn=6)
print(f"Het:{eb_het}")
print(f"Hom:{eb_hom}")

In [ ]:
from scipy.optimize import fsolve

g_sf_dict = dict(zip(g_ca, sf))
def cal_theo_eb_hom(g_ca):
    ## THE ITERATION PARAMETERs
    #physical parameters
    delta = 0.012 #interface width
    Lambda = jnp.sqrt(2)*delta/4
    N = 256

    gamma_ca = g_ca #interface energy between components A,B
    gamma_bc = 1e-2
    gamma_ab = 1e-2
    if gamma_ca > 1e-2:
        Ac = (9*gamma_ca + 12*(gamma_bc - gamma_ab))/(4*jnp.sqrt(2)*Lambda)
        Aa = (9*gamma_ca - 12*(gamma_bc - gamma_ab))/(4*jnp.sqrt(2)*Lambda)
        Ab = 3*jnp.sqrt(2)*gamma_bc/Lambda - 4*Ac/3 - Aa/3 
    else:
        Ab = jnp.sqrt(2)*(9*gamma_bc + 12*(gamma_ab - gamma_ca))/(8*Lambda)
        Ac = jnp.sqrt(2)*(9*gamma_bc - 12*(gamma_ab - gamma_ca))/(8*Lambda)
        Aa = 3*jnp.sqrt(2)*gamma_ab/Lambda - 4*Ab/3 - Ac/3
    A = jnp.array([Aa, Ab, Ac])
    C = jnp.max(A + jnp.roll(A, 1))/6
    print(A)

    rc = 0.1 # the final radius of conponent C
    eps = 1e-14 
    def ChemicalPotential(i, ua, ub, uc):
        P = jnp.power(1-ua, 2)*jnp.power(1-ub, 2)*jnp.power(1-uc, 2)
        S = jnp.power(ua, 2)+jnp.power(ub, 2)+jnp.power(uc, 2)
        u = [ua, ub, uc]
        mu_i = 2*A[i]*u[i]*(1 - u[i])*(1 - 2*u[i]) + 2*C*(u[i]*(S - u[i]**2) - (1-u[i])*(P + eps)/(jnp.power(1-u[i], 2) + eps))
        return mu_i

    def energy(ua, ub, uc):
        bulk_term = Aa*jnp.power(ua, 2)*jnp.power(ua-1, 2) + Ab*jnp.power(ub, 2)*jnp.power(ub-1, 2) + Ac*jnp.power(uc, 2)*jnp.power(uc-1, 2) + C*(jnp.power(ua,2)*jnp.power(ub, 2) + jnp.power(ua,2)*jnp.power(uc,2) + jnp.power(uc,2)*jnp.power(ub,2) + jnp.power(1-ua,2)*jnp.power(1-ub,2)*jnp.power(1-uc,2))
        return bulk_term
    
    def get_mean_uc(uc0):
        assert uc0.shape == (N, N), "shape wrong"
        c = []
        for i in range(10, N, 10):
            c.append(np.mean(uc0[i, :]))
        return sum(c)/len(c)
    
    def get_mean_ua(ua0):
        assert ua0.shape == (N, N), "shape wrong"
        c = []
        for i in range(150, 240, 5):
            c.append(np.mean(ua0[i, :]))
        return sum(c)/len(c)

    def get_mean_ub(ub0):
        assert ub0.shape == (N, N), "shape wrong"
        c = []
        for i in range(150, 240, 5):
            c.append(np.mean(ub0[i, :]))
        return sum(c)/len(c)
    
    def get_mean_ua2(ua0):
        assert ua0.shape == (N, N), "shape wrong"
        c = []
        for i in range(20, 110, 5):
            c.append(np.mean(ua0[i, :]))
        return sum(c)/len(c)

    def get_mean_ub2(ub0):
        assert ub0.shape == (N, N), "shape wrong"
        c = []
        for i in range(20, 110, 5):
            c.append(np.mean(ub0[i, :]))
        return sum(c)/len(c)

    def critical_Delta_F(gca):
        u = np.load(f"Nucleation_ZTS/TernaryMix/2d/Output/AsymmetricCase/Data-InterfaceWidth-0.012/gamma_ca-{gamma_ca}/homogeneous/concentration.npy")
        if gca>1e-2: 
            
            ua0 = u[0, 0, :, :]
            ub0 = u[1, 0, :, :]
            uc0 = u[2, 0, :, :]
            mean_uc = get_mean_uc(uc0)
            mean_ua = get_mean_ua2(ua0)
            mean_ub = get_mean_ub2(ub0)
            print(f"case gamma_ac = {gamma_ca}, mean_ua = {mean_ua}, mean_ub = {mean_ub}, mean_uc = {mean_uc}, sum {mean_ua+ mean_ub + mean_uc}")

            ua0 = mean_ua
            ub0 = mean_ub
            uc0 = mean_uc

            ua1 = 0
            ub1 = 0
            uc1 = 1

            ua2 = 0
            ub2 = 1
            uc2 = 0

            f0 = energy(ua=ua0, ub=ub0, uc=uc0) 
            f1 = energy(ua=ua1, ub=ub1, uc=uc1)

            mu_a0 = ChemicalPotential(0, ua=ua0, ub=ub0, uc=uc0)
            mu_b0 = ChemicalPotential(1, ua=ua0, ub=ub0, uc=uc0)
            mu_c0 = ChemicalPotential(2, ua=ua0, ub=ub0, uc=uc0)
            
            df = f1 - f0
            k = (ua2 - ua1)*mu_a0 + (ub2 - ub1)*mu_b0 + (uc2 - uc1)*mu_c0
            def equation(x):
                return (df + k) * x - 4 * jnp.pi * k * x**3 + gamma_bc
            Rc = fsolve(equation, 0)
            
            DFc = (f1 - f0)*jnp.pi*Rc**2 + ((ua2 - ua1)*mu_a0 +  (ub2 - ub1)*mu_b0 + (uc2 - uc1)*mu_c0)*(jnp.pi*Rc**2)*(1 - 2*jnp.pi*Rc**2) + 2*jnp.pi*Rc*gamma_bc
            return Rc, DFc
        else: 
            
            ua0 = u[0, 0, :, :]
            ub0 = u[1, 0, :, :]
            uc0 = u[2, 0, :, :]
            mean_uc = get_mean_uc(uc0)
            mean_ua = get_mean_ua(ua0)
            mean_ub = get_mean_ub(ub0)
            print(f"case gamma_ac = {gamma_ca}, mean_ua = {mean_ua}, mean_ub = {mean_ub}, mean_uc = {mean_uc}, sum {mean_ua+ mean_ub + mean_uc}")

            ua0 = mean_ua
            ub0 = mean_ub
            uc0 = mean_uc

            ua1 = 0
            ub1 = 0
            uc1 = 1

            ua2 = 1
            ub2 = 0
            uc2 = 0

            f0 = energy(ua=ua0, ub=ub0, uc=uc0) 
            f1 = energy(ua=ua1, ub=ub1, uc=uc1)

            mu_a0 = ChemicalPotential(0, ua=ua0, ub=ub0, uc=uc0)
            mu_b0 = ChemicalPotential(1, ua=ua0, ub=ub0, uc=uc0)
            mu_c0 = ChemicalPotential(2, ua=ua0, ub=ub0, uc=uc0)
            
            df = f1 - f0
            k = (ua2 - ua1)*mu_a0 + (ub2 - ub1)*mu_b0 + (uc2 - uc1)*mu_c0
            def equation(x):
                return (df + k) * x - 4 * jnp.pi * k * x**3 + gamma_ca
            Rc = fsolve(equation, 0)
            
            DFc = (f1 - f0)*jnp.pi*Rc**2 + ((ua2 - ua1)*mu_a0 +  (ub2 - ub1)*mu_b0 + (uc2 - uc1)*mu_c0)*(jnp.pi*Rc**2)*(1 - 2*jnp.pi*Rc**2) + 2*jnp.pi*Rc*gamma_ca
            return Rc, DFc

    theo_r, theo_eb_hom = critical_Delta_F(gca=g_ca)


    return float(theo_r), float(theo_eb_hom)




theo_hom = []
theo_het = []
theo_cr_hom = []
theo_cr_het = []
for gamma_ca in tqdm(g_ca):
    theo_r, theo_eb_hom = cal_theo_eb_hom(g_ca=gamma_ca)
    theo_eb_het = theo_eb_hom*g_sf_dict[gamma_ca]
    theo_hom.append(theo_eb_hom)
    theo_het.append(theo_eb_het)
    theo_cr_hom.append(theo_r)

print(f"Theo Het:{theo_het}")
print(f"Theo Hom:{theo_hom}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.pylab as pylabs
from matplotlib.ticker import ScalarFormatter
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Set1.colors)

myparams = {

   'axes.labelsize': '13',

   'xtick.labelsize': '11',

   'ytick.labelsize': '11',

   'lines.linewidth': 1,

   'legend.fontsize': '10',

   'font.family': 'Times New Roman',

   'figure.figsize': '9, 4' 

}
pylabs.rcParams.update(myparams)  

plt.figure(dpi=100)

plt.subplot(1,2,1)

plt.plot(g_ca, theo_hom, 'o--',  markersize=4, label='Theoretical $E_{b}^{hom}$')
plt.plot(g_ca, eb_hom, 's-',  markersize=6, linewidth=0.7, label='Experimental $E_{b}^{hom}$')
plt.plot(g_ca, theo_het, 'x--',  markersize=4, label='Theoretical $E_{b}^{het}$')
plt.plot(g_ca, eb_het, 'v-',  markersize=6, linewidth=0.7, label='Experimental $E_{b}^{het}$')

plt.xlabel('Interface Energy $\\gamma_{ac}$')
plt.ylabel('Energy Barrier ($E_{b}^{hom / het}$)')
plt.title("Energy Barriers", fontsize=14)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

plt.legend(loc='upper left', frameon=True)
plt.grid(True, linestyle='--', linewidth=0.5)

plt.yscale('log', base=10)
plt.xscale('log', base=10)
#plt.ylim(0, 0.8)

interface_energy_vector = np.array(g_ca)
numerical_shape_factor_vector = np.array(eb_het)/np.array(eb_hom)
theoretical_shape_factor_vector = np.array(sf)

plt.subplot(1,2,2)
plt.plot(interface_energy_vector, theoretical_shape_factor_vector,  "o--", markersize=4, label='Theoretical shape factor')
plt.plot(interface_energy_vector, numerical_shape_factor_vector, "s-", linewidth=0.7, markersize=6, label='Numerical shape factor')
plt.legend(fontsize=12)
plt.title("Shape factor", fontsize=14)
plt.xlabel('Interface Energy $\\gamma_{ac}$')
plt.ylabel('Shape Factor')
plt.grid(True, linestyle='--', linewidth=0.5)
#plt.ylim(0.1, 0.7)

plt.tight_layout()
plt.savefig("/home/ms/akrito/string-method-nucleation/correction/2d/nonsym/results.pdf", dpi=100, bbox_inches='tight')


relative_err = np.abs(numerical_shape_factor_vector - theoretical_shape_factor_vector)/theoretical_shape_factor_vector
print(f"low:{np.min(relative_err)}\n high:{np.max(relative_err)}")
print(relative_err)